In [1]:
!pip install biopython
# lib libraries
import time
# 3rd-party libraries
import Bio
from Bio import Align
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 8.5 MB/s 


For discussion of isoforms, see https://en.wikipedia.org/wiki/Protein_isoform

In [2]:
class AlignmentFailedError(Exception):
    pass

In [3]:
# see https://rest.uniprot.org/docs/#/
BASE_QUERY = "https://rest.uniprot.org/uniprotkb/search?query=accession%3D"
def get_protein(acc_num: str) -> dict:
    '''Get the information in UniProt associated with accession number
    acc_num'''
    resp = requests.get(BASE_QUERY + acc_num)
    resp.raise_for_status()
    return resp.json()['results']

In [4]:
def get_sequence(prot: dict) -> str:
    return prot[0]['sequence']['value']

In [5]:
def get_isoform_ids(prot: dict) -> list:
    xrefs = prot[0]['uniProtKBCrossReferences']
    out = set()
    for ref in xrefs:
        isof = ref.get('isoformId')
        if isof is not None:
            out.add(isof)
    return sorted(out)

In [6]:
def get_isoforms(prot: dict) -> dict:
    iso_ids = get_isoform_ids(prot)
    seqs = {}
    for id_ in iso_ids:
        try:
            seqs[id_] = get_protein(id_)
        except:
            continue
    return seqs

In [7]:
def get_all_prots(acc_num: str) -> dict:
    prot = get_protein(acc_num)
    prots = {acc_num: prot}
    seq = get_sequence(prot)
    isos = get_isoforms(prot)
    # remove the isoforms with the same sequence as the base acc num
    for iso_acc_num, iso in list(isos.items()):
        iso_seq = get_sequence(iso)
        if iso_acc_num != acc_num and iso_seq == seq:
            del isos[iso_acc_num]
    return {**prots, **isos}

In [8]:
claudin = 'P56856'
all_prots = get_all_prots(claudin)

In [9]:
def get_all_seqs(prots: dict) -> dict:
    return {k: get_sequence(v) for k, v in prots.items()}

In [10]:
seqs = get_all_seqs(all_prots)
seq1, seq2 = seqs.values()

In [11]:
def get_aligner(**kwargs) -> Bio.Align.PairwiseAligner:
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.mismatch_score = kwargs.get('mismatch_score', -10)
    aligner.target_internal_open_gap_score = kwargs.get('target_internal_open_gap_score', -20.)
    aligner.target_internal_extend_gap_score = kwargs.get('target_internal_extend_gap_score', -5.)
    aligner.target_left_open_gap_score = kwargs.get('target_left_open_gap_score', -10.)
    aligner.target_left_extend_gap_score = kwargs.get('target_left_extend_gap_score', -0.5)
    aligner.target_right_open_gap_score = kwargs.get('target_right_open_gap_score', -10.)
    aligner.target_right_extend_gap_score = kwargs.get('target_right_extend_gap_score', -0.5)
    aligner.query_internal_open_gap_score = kwargs.get('query_internal_open_gap_score', -10.)
    aligner.query_internal_extend_gap_score = kwargs.get('query_internal_extend_gap_score', -0.5)
    aligner.query_left_open_gap_score = kwargs.get('query_left_open_gap_score', -10.)
    aligner.query_left_extend_gap_score = kwargs.get('query_left_extend_gap_score', -0.5)
    aligner.query_right_open_gap_score = kwargs.get('query_right_open_gap_score', -10.)
    aligner.query_right_extend_gap_score = kwargs.get('query_right_extend_gap_score', -0.5)
    return aligner

In [12]:
aligner = get_aligner()
alignments = aligner.align(seq1, seq2)
print(alignments[0])

MSTTTCQVVAFLLSILGLAGCIAATGMDMWSTQDLYDNPVTSVFQYEGLWRSCVRQSSGFTECRPYFTILGLPAMLQAVRALMIVGIVLGAIGLLVSIFALKCIRIGSMEDSAKANMTLTSGIMFIVSGLCAIAGVSVFANMLVTNFWMSTANMYTGMGGMVQTVQTRYTFGAALFVGWVAGGLTLIGGVMMCIACRGLAPEETNYKAVSYHASGHSVAYKPGGFKASTGFGSNTKNKKIYDGGARTEDEVQSYPSKHDYV
                                                                     ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
MAVTACQGLGFVVSLIGIAGIIAATCMDQWSTQDLYNNPVTAVFNYQGLWRSCVRESSGFTECRGYFTLLGLPAMLQAVRALMIVGIVLGAIGLLVSIFALKCIRIGSMEDSAKANMTLTSGIMFIVSGLCAIAGVSVFANMLVTNFWMSTANMYTGMGGMVQTVQTRYTFGAALFVGWVAGGLTLIGGVMMCIACRGLAPEETNYKAVSYHASGHSVAYKPGGFKASTGFGSNTKNKKIYDGGARTEDEVQSYPSKHDYV



In [13]:
def to_fasta(seqs: dict) -> str:
    fasta = ''
    for acc_num, seq in seqs.items():
        seqrec = SeqRecord(Seq(seq))
        seqrec.id = acc_num
        fasta += seqrec.format('fasta')
    return fasta

In [31]:
# https://rest.uniprot.org/beta/docs/
WEBSITE_API = "https://rest.uniprot.org/beta"
def request_multi_alignment(seqs: dict) -> str:
    '''send a request to the European Bioinformatics Institute
    for multiple alignment of several sequences'''
    fasta = to_fasta(seqs)
    r = requests.post(
        "https://www.ebi.ac.uk/Tools/services/rest/clustalo/run", 
        data={
            "email": "lestimpe@gmail.com",
            "iterations": 1,
            "outfmt": "clustal_num",
            "order": "aligned",
            "sequence": fasta
           }
    )
    r.raise_for_status()
    job_id = r.text
    job_status = 'RUNNING'
    # ping the server every few seconds to see if the job is done
    while job_status == 'RUNNING':
        time.sleep(4)
        job_status_req = requests.get(
            f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}")
        job_status_req.raise_for_status()
        job_status = job_status_req.text
    # now that the job is done, get the alignment
    resp = requests.get(
        f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/aln-clustal_num")
    resp.raise_for_status()
    return resp.text

In [32]:
def align_isoforms(acc_num: str, **kwargs) -> str:
    '''get all isoforms of the protein with accession number acc_num,
    and return a sequence alignment.
    Can also pass in keyword arguments to set parameters for the alignment,
    if there are only two isoforms to align'''
    prots = get_all_prots(acc_num)
    seqs = get_all_seqs(prots)
    if len(seqs) < 2:
        print(f"The protein with UniProt accession number {acc_num} has only one isoform")
        (acc1, seq1) = list(seqs.items())[0]
        return seq1
    if len(seqs) == 2:
        # BioPython can do a simple alignment for us
        aligner = get_aligner(**kwargs)
        (acc1, seq1), (acc2, seq2) = seqs.items()
        alignments = aligner.align(seq1, seq2)
        line1, line2, line3 = str(alignments[0]).splitlines()
        return f'{acc1:<12}{line1}\n{" "*12}{line2}\n{acc2:<12}{line3}'
    # for more than three isoforms, we need to ask EBI for help
    return request_multi_alignment(seqs)

In [16]:
# test aligning something with two isoforms
alignment = align_isoforms(claudin)
print(alignment)

P56856      MSTTTCQVVAFLLSILGLAGCIAATGMDMWSTQDLYDNPVTSVFQYEGLWRSCVRQSSGFTECRPYFTILGLPAMLQAVRALMIVGIVLGAIGLLVSIFALKCIRIGSMEDSAKANMTLTSGIMFIVSGLCAIAGVSVFANMLVTNFWMSTANMYTGMGGMVQTVQTRYTFGAALFVGWVAGGLTLIGGVMMCIACRGLAPEETNYKAVSYHASGHSVAYKPGGFKASTGFGSNTKNKKIYDGGARTEDEVQSYPSKHDYV
                                                                                 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
P56856-2    MAVTACQGLGFVVSLIGIAGIIAATCMDQWSTQDLYNNPVTAVFNYQGLWRSCVRESSGFTECRGYFTLLGLPAMLQAVRALMIVGIVLGAIGLLVSIFALKCIRIGSMEDSAKANMTLTSGIMFIVSGLCAIAGVSVFANMLVTNFWMSTANMYTGMGGMVQTVQTRYTFGAALFVGWVAGGLTLIGGVMMCIACRGLAPEETNYKAVSYHASGHSVAYKPGGFKASTGFGSNTKNKKIYDGGARTEDEVQSYPSKHDYV


In [33]:
# test aligning things with >= 3 isoforms
ampk_gamma = 'P54619' # https://en.wikipedia.org/wiki/PRKAG1
align_ampk = align_isoforms(ampk_gamma)
print(align_ampk)

CLUSTAL O(1.2.4) multiple sequence alignment


P54619        METVISSDSSPAVENEHPQETPESNNSVYTSFMKSHRCYDLIPTSSKLVVFDTSLQVKKA	60
P54619-2      --------------------------------MKSHRCYDLIPTSSKLVVFDTSLQVKKA	28
P54619-3      METVISSDSSPAVENEHPQETPESNNSVYTSFMKSHRCYDLIPTSSKLVVFDTSLQVKKA	60
                                              ****************************

P54619        FFALVTNGVRAAPLWDSKKQSFV---------GMLTITDFINILHRYYKSALVQIYELEE	111
P54619-2      FFALVTNGVRAAPLWDSKKQSFV---------GMLTITDFINILHRYYKSALVQIYELEE	79
P54619-3      FFALVTNGVRAAPLWDSKKQSFVVLRALSCPLGMLTITDFINILHRYYKSALVQIYELEE	120
              ***********************         ****************************

P54619        HKIETWREVYLQDSFKPLVCISPNASLFDAVSSLIRNKIHRLPVIDPESGNTLYILTHKR	171
P54619-2      HKIETWREVYLQDSFKPLVCISPNASLFDAVSSLIRNKIHRLPVIDPESGNTLYILTHKR	139
P54619-3      HKIETWREVYLQDSFKPLVCISPNASLFDAVSSLIRNKIHRLPVIDPESGNTLYILTHKR	180
              ************************************************************

P54619        ILKF

In [34]:
# test aligning things with only one isoform
ckb = 'P12277' # https://en.wikipedia.org/wiki/CKB_(gene)
align_ckb = align_isoforms(ckb)
print(align_ckb)

The protein with UniProt accession number P12277 has only one isoform
MPFSNSHNALKLRFPAEDEFPDLSAHNNHMAKVLTPELYAELRAKSTPSGFTLDDVIQTGVDNPGHPYIMTVGCVAGDEESYEVFKDLFDPIIEDRHGGYKPSDEHKTDLNPDNLQGGDDLDPNYVLSSRVRTGRSIRGFCLPPHCSRGERRAIEKLAVEALSSLDGDLAGRYYALKSMTEAEQQQLIDDHFLFDKPVSPLLLASGMARDWPDARGIWHNDNKTFLVWVNEEDHLRVISMQKGGNMKEVFTRFCTGLTQIETLFKSKDYEFMWNPHLGYILTCPSNLGTGLRAGVHIKLPNLGKHEKFSEVLKRLRLQKRGTGGVDTAAVGGVFDVSNADRLGFSEVELVQMVVDGVKLLIEMEQRLEQGQAIDDLMPAQK


In [35]:
has1 = 'Q92839' # https://en.wikipedia.org/wiki/HAS1
align_has1 = align_isoforms(has1)

The protein with UniProt accession number Q92839 has only one isoform
